# Libraries and modules


In [1]:
import pandas as pd 
from datetime import datetime
from ML.internal import selectdfQ, regexdata as rd, scrapedInternalPage
from ML.tools import savedata as sd, deltempfiles as dtf
from ML.directories import master_directory as md, directory_bat_file 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# __ENVIRONMENT__

In [2]:
# Variables 

mth = 'FEB'       #Update each month
yr = '2024'       #Update each month

op = 'Sale'
web = 'ML'

#Master directory
master_dir = md()
master_dir = f'{master_dir}/{web}'


#IMPORT
#Directories to import files 
moni_dir_imp = f'{master_dir}/FRONTPAGE/MONITORING/{yr}/{mth}'

#import files 
###############################################################################################################
fl_master_imp = f'{web}_{mth}{yr}_{op}_Front.csv'

inputfile = f'{moni_dir_imp}/{fl_master_imp}' 



#EXPORT
#Directories to export files 
result_dir = f'{master_dir}/INTERNAL/JOIN'

#Export files 
###############################################################################################################
#This files is going to be joined for create the master file, which contains all the data for the fron page
fl_pre = f'{web}_join_{op}INT_{mth}{yr}_'


# Ruta al archivo .bat
ruta_bat = directory_bat_file()
ruta_bat = f'{ruta_bat}/deltemporal_exect.bat'

# Read frontpage df 

In [3]:
df = pd.read_csv(f'{inputfile}', sep = ';', encoding = 'utf-16')
len(df)

C:\Users\luisG\AppData\Local\Temp\ipykernel_46540\176173939.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{inputfile}', sep = ';', encoding = 'utf-16')


57245

## Choosing the df 


In [4]:
dfQ2, _ = selectdfQ(df, 2, 'venta') # 1 only select the first 25% of the data 

56919
Usted seleccionó dfQ2 con: 28460 obs


# Split data 

In [5]:
#Separación Q1
#############################################################################################################
# Calcula cuántos DataFrames de 5000 observaciones necesitarás
num_dfs = len(dfQ2) // 500 + (1 if len(dfQ2) % 500 != 0 else 0)
# Dividir df_inmobiliaria entre 5000 observaciones cada uno para hacer el scrapign sin problemas
dfs = {f'{fl_pre}_Q2_{i}': dfQ2.iloc[i:i+500] for i in range(0, len(dfQ2), 500)}
# Nombres de df_inmobiliaria
print(f'Se crearon {num_dfs} DataFrames para df_inmobiliaria:')

Se crearon 57 DataFrames para df_inmobiliaria:


# __Scraping__

In [ ]:
inmueble_slow_resultados = []    

try:
    start_time = datetime.now()
    print(f"El proceso de scraping ha comenzado a las {start_time.strftime('%H:%M')} del {start_time.strftime('%d/%m/%Y')}")

    for df_key, df_value in dfs.items():
        print(f"Realizando scraping en {df_key}...")
        scraped_data = scrapedInternalPage(df_value)
        inmueble_slow_resultados.extend(scraped_data) 
        processed_data = rd(scraped_data)
        sd(processed_data, f'{result_dir}/{df_key}')
        dtf(ruta_bat)

    #Fin del scraping: 
    end_time = datetime.now()
    print(f"El proceso de scraping ha terminado a las {end_time.strftime('%H:%M')} del {end_time.strftime('%d/%m/%Y')}")
    
except Exception as e:
    print(f"Se produjo un error en el scraping: {e}")
